## Question Generation

In [6]:
# collect all paragraphs
path = '../data/cast19/test_paragraphs.tsv'
qp_ids = {}
paras = {}
with open(path, encoding='utf-8') as f:
    for i, line in enumerate(f):
        q_id, para_id, para_text = line.rstrip().split('\t')
        if q_id not in qp_ids:
            qp_ids[q_id] = []
        qp_ids[q_id].append(para_id)
        if para_id not in paras:
            paras[para_id] = para_text
print(len(paras))
print(qp_ids)

6485
{'31_1': ['CAR_1463f964653c5c9f614a0a88d26b175e4a8120f1', 'CAR_172e16e89ea3d5546e53384a27c3be299bcfe968', 'CAR_2174ad0aa50712ff24035c23f59a3c2b43267650', 'CAR_25a576af9caa6422f55c2acf945dc79b423fb41e', 'CAR_2dc597ac2fc10917a752552bc335e6ac1aedc3f0', 'CAR_3249e5618575a849152c02b05f4fda924f10326f', 'CAR_393cb4e18a9d30018e843c4d37c564272ec5fa6f', 'CAR_40c64256e988c8103550008f4e9b7ce436d9536d', 'CAR_41ab200340a8fcae748b366ff0b0506e6b87cefa', 'CAR_41b7dce4f8a72ee34d78c2b5c363272a54997f27', 'CAR_55f68f0a33f49f6015035f70d8685e293389c9d6', 'CAR_5b50a00d9e1bd1d4ac3a973a656c1b20ab96ec1e', 'CAR_683d2822b14dca56ef1ffd93c997258c3b2cc083', 'CAR_71fe664149cf06de1fcbb55dfb239f48687753c0', 'CAR_7b85fdb20fa85b01050fc49656cbc548d231defe', 'CAR_87d4034ad40c274e7b3a40ab7cfe5aadc2fd214c', 'CAR_8b8521b6835068974cc26ff1dfb3d76f84b7a504', 'CAR_bc285835e2ded639fa3ca767f43c8ed4bbbbdfe6', 'CAR_d37891cd657001a45ae65a1864b6af96a9b4543c', 'CAR_dbeb8cd6edbddddd97b665073523548470dffe44', 'CAR_eed3d1eb3efc8660a010

In [7]:
import datetime
import json
import os

squash_path = '/home/svakule/squash-generation'

top_p = 0.9
gen_frac = 0.5
spec_frac = 0.8


for q in qp_ids:
    key = q
    ps = []
    for p in qp_ids[q]:
        ps.append(paras[p])
    
    metadata = {
            "input_text": "\n".join(ps),
            "key": key,
            "timestamp": str(datetime.datetime.now()),
            "settings": {
                "top_p": top_p,
                "gen_frac": gen_frac,
                "spec_frac": spec_frac
            }
        }

    os.mkdir("%s/squash/temp/trec_cast19/%s" % (squash_path, key))

    with open('%s/squash/temp/trec_cast19/%s/metadata.json' % (squash_path, key), 'w') as f:
        f.write(json.dumps(metadata))

In [3]:
# for d in squash/temp/trec_cast19/*/ ; do
#     k="$(basename $d)"
#     echo "$k"
#     python squash/extract_answers.py --key trec_cast19/$k
#     python question-generation/interact.py --model_checkpoint question-generation/gpt2_corefs_question_generation --model_type gpt2 --key trec_cast19/$k --filename squash/temp/trec_cast19/$d/input.pkl
# done
# vim squash/temp/trec_cast19/generated_questions.json

## Question Evaluation

In [11]:
# collect all GS questions: independent
path_qs = '../data/cast19/human_questions.txt'

gs_qs = {}
with open(path_qs, 'r') as f:
    next(f)
    for line in f:
        q_id, q = line.split('\t')
#         print(q_id)
        gs_qs[q_id] = (q.strip('\n'))
print(len(gs_qs))
print(gs_qs['1_11'])

748
What is the fastest way to become a nurse practitioner NP?


In [15]:
import os
import json

sim_threshold = 0.8  # rouge similarity threshold for duplicate questions

squash_path = '/home/svakule/squash-generation/squash/temp/trec_cast19'

for question_id in os.listdir(squash_path):
    print(question_id)
    with open(squash_path+'/%s/generated_questions.json' % question_id) as f:
        paragraphs = json.load(f)['data'][0]['paragraphs']
    #     print(len(paragraphs))
        for i, p in enumerate(paragraphs):
            print(gs_qs[question_id])  # original (reference) question
            print(p['context']) # original paragraph
            q_class = None
            qs = []
            for qa in p['qas']:
                if q_class != qa['class']:
                    q_class = qa['class']
                    print('\n')
                    print(qa['class'])
                # do not keep duplicate questions for the same paragraph
                q = qa['question'].lower()
                if q not in qs:
                    # check similarity to existing questions
                    duplicate = False
                    for q_ref in qs:
    #                     print(q_ref)
                        scores = scorer.score(q_ref, q)
                        rouge1f = scores['rouge1'].fmeasure
                        if rouge1f > sim_threshold:
                            duplicate = True
                            break
    #                     print(rouge1f)
                    if not duplicate:
                        qs.append(q)
                        print("Q:", qa['question'])  # generated (predicted) question
                        print("A:", qa['answers'][0]['text'])
            print('\n')
    break



67_6
What are the symptoms of anemia?
Triosephosphate isomerase deficiency is a disorder characterized by a shortage of red blood cells (anemia), movement problems, increased susceptibility to infection, and muscle weakness that can affect breathing and heart function. The anemia in this condition begins in infancy. Since the anemia results from the premature breakdown of red blood cells (hemolysis), it is known as hemolytic anemia. A shortage of red blood cells to carry oxygen throughout the body leads to extreme tiredness (fatigue), pale skin (pallor), and shortness of breath. When the red cells are broken down, iron and a molecule called bilirubin are released; individuals with triosephosphate isomerase deficiency have an excess of these substances circulating in the blood. Excess bilirubin in the blood causes jaundice, which is a yellowing of the skin and the whites of the eyes. Movement problems typically become apparent by age 2 in people with triosephosphate isomerase deficiency

Q: What is the  point mutation?
A: The point mutation does not affect the catalysis rate, but rather, affects the assembly of the enzyme into a homodimer.
Q: When does it become apparent to people with triosephosphate deficiency that muscle weakness is due to deficiency of triosephosphate isomerase deficiency?
A: age 2
Q: What is the abbreviation for triosephosphate isomerase deficiency?
A: TPI
Q: What enzyme is deficient in triosephosphate isomerase deficiency a deficiency?
A: DHAP


What are the symptoms of anemia?
Anemia, also spelled anaemia, is usually defined as a decrease in the total amount of  red blood cell s (RBCs) or  hemoglobin  in the  blood. It can also be defined as a lowered ability of the blood to carry  oxygen. When anemia comes on slowly, the symptoms are often vague and may include  feeling tired, weakness,  shortness of breath  or a poor ability to exercise. Anemia that comes on quickly often has greater symptoms, which may include confusion,  feeling like one is 

Q: What does pernicious anemia cause?
A: Symptoms
Q: What is a term that describes anemia in the body?
A: PA
Q: What type of anemia can cause symptoms of?
A: Anemia


What are the symptoms of anemia?
Anemia goes undetected in many people and symptoms can be minor. The symptoms can be related to an underlying cause or the anemia itself.Most commonly, people with anemia report feelings of  weakness  or  tired, and sometimes poor concentration. They may also report  shortness of breath  on exertion.  In very severe anemia, the body may compensate for the lack of oxygen-carrying capability of the blood by increasing  cardiac output.  The patient may have symptoms related to this, such as  palpitation s,  angina  (if pre-existing heart disease is present), intermittent  claudication  of the legs, and symptoms of  heart failure.On examination, the signs exhibited may include  pallor  ( pale   skin, lining  mucosa,  conjunctiva  and  nail beds ), but this is not a reliable sign.  There may be

Q: What is the term for Blood Disorders Affecting Red Blood Cells?
A: Blood Disorders Affecting Red Blood Cells.
Q: What are blood disorders that affect red blood cells?
A: Blood disorders that affect red blood cells include:
Q: What is anemia called that affects red blood cells?
A: Anemia:
Q: What are people with anemia with blood disorders?
A: People with anemia have a low number of red blood cells.
Q: What is one example of anemia that affects red blood cells?
A: Mild anemia often causes no symptoms.
Q: What are some other symptoms of anemia?
A: More severe anemia can cause fatigue, pale skin, and shortness of breath with exertion.
Q: What does iron make?
A: Iron is necessary for the body to make red blood cells.
Q: How many causes are there of anemia?
A: Low iron intake and loss of blood due to menstruation are the most common causes of iron-deficiency anemia.
Q: Any other causes?
A: It may also be caused by blood loss from the GI tract because of ulcers or cancer.
Q: What part of 

Q: What is one symptom of anemia?
A: Mild anemia often causes no symptoms.
Q: What is the name of an iron deficiency?
A: Iron-deficiency anemia:
Q: What type of diseases affect red blood cells?
A: Blood disorders that affect red blood cells include:
Q: What is anemia that affects red blood cells called?
A: Anemia:
Q: What kind of symptoms does Mild anemia cause?
A: Mild anemia often causes no symptoms.
Q: What's anemia?
A: More severe anemia can cause fatigue, pale skin, and shortness of breath with exertion.


What are the symptoms of anemia?
Blood disorders that affect red blood cells include: Anemia: People with anemia have a low number of red blood cells. Mild anemia often causes no symptoms. More severe anemia can cause fatigue, pale skin, and shortness of breath with exertion.


general
Q: What is anemia?
A: Blood disorders that affect red blood cells include:
Q: What are people with anemia?
A: People with anemia have a low number of red blood cells.
Q: What is a mild anemia?
A: 

...

A: People with anemia have a low number of red blood cells.
Q: What is anemia?
A: Mild anemia often causes no symptoms.
Q: What else can anemia cause?
A: More severe anemia can cause fatigue, pale skin, and shortness of breath with exertion.
Q: What is the main cause of iron deficiency?
A: Iron-deficiency anemia:
Q: Why is that?
A: Iron is necessary for the body to make red blood cells.


specific
Q: What kind of disorders can cause anemia?
A: Blood disorders that affect red blood cells include:
Q: What type of disease can cause low red blood cells?
A: Anemia :
Q: What is the name of a person with anemia?
A: People with anemia have a low number of red blood cells.
Q: What is a symptom that causes anemia?
A: Mild anemia often causes no symptoms.
Q: What type of anemia is it?
A: Iron-deficiency anemia:
Q: What does iron make?
A: Iron is necessary for the body to make red blood cells.


What are the symptoms of anemia?
Symptoms include tachycardia, hypotension, lethargy, and weakness. Ane

Q: What is the number of yellow urine?
A: 5  
Q: What is the term for anemia that is caused by chronic red blood cell destruction?
A: Caused
Q: How many yellow or red urine are there?
A: 2  
Q: How many symptoms of anemia are there?
A: 3  


What are the symptoms of anemia?
Anemia also can occur if your red blood cells don't contain enough hemoglobin (HEE-muh-glow-bin). Hemoglobin is an iron-rich protein that gives blood its red color. This protein helps red blood cells carry oxygen from the lungs to the rest of the body. If you have anemia, your body doesn't get enough oxygen-rich blood. As a result, you may feel tired or weak. You also may have other symptoms, such as shortness of breath, dizziness, or headaches. Severe or long-lasting anemia can damage your heart, brain, and other organs in your body. Very severe anemia may even cause death. Blood is made up of many parts, including red blood cells, white blood cells, platelets (PLATE-lets), and plasma (the fluid portion of blood).


Q: How many symptoms do people have that can cause severe anemia?
A: 5
Q: How many shortness of breath symptoms does someone have?
A: 6  


What are the symptoms of anemia?
Anemia is a lower than normal red blood cell count or the lack of hemoglobin in your red blood cells, resulting in a wide range of symptoms. Anemia can cause fatigue, headaches, shortness of breath, dizziness, rapid heartbeat, and a number of other symptoms. There are several different types of anemia, such as dietary deficiency anemia, pernicious anemia and megaloblastic anemia.


general
Q: What is anemia?
A: Anemia is a lower than normal red blood cell count or the lack of hemoglobin in your red blood cells, resulting in a wide range of symptoms.
Q: What are the symptoms of anemia?
A: Anemia can cause fatigue, headaches, shortness of breath, dizziness, rapid heartbeat, and a number of other symptoms.


specific
Q: What does that mean?
A: Anemia is a lower than normal red blood cell count or the lack of hemoglobin

Q: What are the main symptoms?
A: The most common symptom of anemia, regardless of type, is a feeling of fatigue and a lack of energy.
Q: What is the usual symptom of anemia?
A: 2  
Q: What is another?
A: Shortness of breath.


specific
Q: What kind of skin is it?
A: Other common symptoms of anemia may include: 1  Paleness of skin.
Q: How many paleness of skin are there?
A: 2  
Q: What are some of the other symptoms?
A: Fast or irregular heartbeat.
Q: How many other symptoms can anemia have?
A: 3  
Q: What other symptoms are listed?
A: Shortness of breath.4  
Q: How many of those are listed?
A: 5  Fever.ymptoms of anemia.
Q: What symptoms does anemia have?
A: The most common symptom of anemia, regardless of type, is a feeling of fatigue and a lack of energy.
Q: How many symptoms are listed?
A: Other common symptoms of anemia may include: 1  Paleness of skin.
Q: How many paleness of skin is a symptom of anemia?
A: 2  
Q: What is another name for anemia?
A: Fast or irregular heartbeat.
Q

Q: What is anemia called that causes tiredness, pale skin, and shortness of breath?
A: Iron-deficiency anemia:
Q: How much is required to make red blood cells?
A: Iron is necessary for the body to make red blood cells.
Q: What is the cause of iron deficiency anemia?
A: Low iron intake and loss of blood due to menstruation are the most common causes of iron-deficiency anemia.lood Disorders Affecting Red Blood Cells.
Q: People with what disease have low red blood cells?
A: Anemia:
Q: How many red blood cells do people have?
A: People with anemia have a low number of red blood cells.
Q: What kind of symptoms does Mild anemia cause?
A: Mild anemia often causes no symptoms.


What are the symptoms of anemia?
Primary myelofibrosis is a condition characterized by the buildup of scar tissue (fibrosis) in the bone marrow, the tissue that produces blood cells. Because of the fibrosis, the bone marrow is unable to make enough normal blood cells. The shortage of blood cells causes many of the sign

In [2]:
# test rouge scorer
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)

scores = scorer.score("what is fsa health plan .", "what is health care fsa plan .")  # reference predicted
print(scores)

{'rouge1': Score(precision=0.8333333333333334, recall=1.0, fmeasure=0.9090909090909091), 'rougeL': Score(precision=0.6666666666666666, recall=0.8, fmeasure=0.7272727272727272)}


In [35]:
# evaluate squash baseline
from collections import defaultdict

q2q = defaultdict(list)

squash_path = '/home/svakule/squash-generation/squash/temp/trec_cast19'

for question_id in os.listdir(squash_path):
    print(question_id)
    try:
        with open(squash_path+'/%s/generated_questions.json' % question_id) as f:
            paragraphs = json.load(f)['data'][0]['paragraphs']
            # group all generated questions to reference questions
            for i, p in enumerate(paragraphs):
                ref_q = gs_qs[question_id]  # original (reference) question
                qs = []
                for qa in p['qas']:
                    # do not keep duplicate questions for the same paragraph
                    q = qa['question']#.lower()
                    if q not in qs:
                        # check similarity to existing questions
                        duplicate = False
                        for q_ref in qs:
        #                     print(q_ref)
                            scores = scorer.score(q_ref, q)
                            rouge1f = scores['rouge1'].fmeasure
                            if rouge1f > sim_threshold:
                                duplicate = True
                                break
        #                     print(rouge1f)
                        if not duplicate:
                            qs.append(q)
                q2q[ref_q].extend(qs)
    except:
        pass

print(len(q2q))       

67_6
79_4
78_3
58_4
59_3
75_2
56_6
32_7
75_8
49_3
69_4
68_9
31_4
54_2
77_6
68_3
54_8
37_2
37_8
34_1
50_7
34_6
37_5
69_3
54_5
31_3
77_1
68_4
69_9
31_9
75_5
56_1
33_7
49_4
58_3
59_4
67_1
79_3
79_9
78_4
61_7
32_11
40_7
61_3
40_3
58_7
79_7
67_5
77_5
68_11
31_7
54_1
69_7
32_4
33_3
56_5
75_1
34_2
34_8
50_3
37_1
67_10
37_6
34_5
50_4
49_7
32_3
56_8
32_9
56_2
33_4
75_6
68_7
77_8
77_2
54_6
67_8
78_7
67_2
59_7
40_4
61_4
59_5
58_8
58_2
79_8
78_5
79_2
61_6
32_10
40_6
34_7
50_6
37_4
68_5
69_8
31_8
69_2
54_4
31_2
49_5
32_1
33_6
75_4
49_2
32_6
33_1
56_7
75_3
49_8
68_2
54_9
77_7
69_5
68_8
31_5
54_3
37_3
50_1
40_1
61_1
78_2
78_8
79_5
67_7
59_2
58_5
59_8
67_3
79_1
78_6
67_9
58_1
59_6
40_5
61_5
37_7
50_5
34_4
32_8
56_3
33_5
32_2
49_6
54_7
31_1
69_1
77_3
68_6
31_6
68_10
69_6
77_4
68_1
33_2
56_4
33_8
32_5
49_1
50_2
34_3
50_8
67_11
61_2
61_8
40_2
40_8
58_6
59_1
67_4
79_6
69_10
78_1
165


In [38]:
i = 0
for q in q2q:
#     print(q)
    print(q2q[q])
    i += 1
    if i > 2:
        break

['What are some of the causes of triosephosphate isomerase deficiency?', 'What is the cause of deficiency?', 'What is hemolytic anemia?', 'What are some symptoms of triosephosphate deficiency?', 'What is bilirubin?', 'What can occur when a person with triosephosphate isomerase deficiency?', 'What is the most common symptom of triosephosphate deficiency?', 'What else is important about the disease?', 'What is the cause of weakness of other muscles?', 'What is seizure?', 'What can cause anemia in triosephosphate isomerase deficiency?', 'What causes diaphragm weakness?', 'Why do people with triosephosphate deficiency develop seizures?', 'What happens to the immune system cells?', 'What are some of the common infections in people with triosephosphate deficiency?', 'What is the most common cause of death in triosephosphate deficiency?', 'What happens to affected individuals without severe nerve damage?', 'How can you tell if you have triosephosphate isomerase deficiency?', 'What does the po

In [39]:
# get questions max similar to GT
import numpy as np

max_scores = []
for q_ref, q_gens in q2q.items():
    all_q_gens_scores = []
    for q in q_gens:
        scores = scorer.score(q_ref, q)  # reference predicted
        rouge1f = scores['rouge1'].fmeasure
#         print(scores['rouge1'].fmeasure)
        all_q_gens_scores.append(scores['rouge1'].fmeasure)
    max_score = max(all_q_gens_scores)
    # show max scoring question
    print(q_ref)
    # show conversation
    print(q_gens[np.argmax(all_q_gens_scores)])
#     print('\n')
    max_scores.append(max_score)
#     break
print(max_scores)
print(np.mean(max_scores))

# specific general specific general

What are the symptoms of anemia?
What are the symptoms of anemia?
What is Herbert Spencer known for?
what is Herbert Spencer famous for
What is ketosis?
what is ketosis?
What are examples of important real-time databases?
What are some examples of real-time databases?
What is the ACL injury?
What is ACL injury?
Where are turkeys from?
Where are turkeys from?
What is the impact of Darwin's theory on modern biology?
What is Darwin's theory of evolution?
Tell me about makos sharks.
What else can you tell me about Mako sharks
Why are turkeys eaten on Thanksgiving?
What are turkey eating on Thanksgiving?
What is the relationship of Netflix with Blockbuster?
What is the brand name of Blockbuster?
What are good melatonin sources in food?
What are some foods that are good sources?
What are typical pasta dishes?
What are some Italian pasta dishes?
What are the symptoms of lung cancer?
What are the symptoms of lung cancer?
Which Smithsonian museums in Washington D.C. are the most popular?
What a

Is throat cancer treatable?
What is Throat cancer?
What are Netflix's other competitors?
What are some other  competitors
What are the different types of sharks?
What are the different types of sharks?
Who was the author and when was Neverending Story film published?
When was The Neverending Storys published?
When and how were turkeys domesticated?
When were turkeys first domesticated?
How did Netflix originally work?
How did Netflix start?
What about for great whites sharks, the largest ever to have lived on Earth or caught?
What is the largest ever great white shark to be recorded?
Tell me about the Neverending Story film.
What else can you tell me about the story?
Compare and contrast microevolution and macroevolution.
what are macroevolution and microevolution?
What was turkeys' importance in native Turkey cultures?
What was the turkey clan?
Tell me about cooking schools and classes in Emilia-Romagna.
What can you tell me about cooking classes?
Tell me about the history of Washingt

# Question Selection

In [40]:
# pick the most frequently occuring questions across parapgraphs
import numpy as np

scores = []
for q_ref, q_gens in q2q.items():
    # select question
    print(q_ref)
    all_q_gens_scores = []
    # compare all questions to each other
    for q in q_gens:
#         print(q)
        sim_sum = 0
        for q2 in q_gens:
            score = scorer.score(q2, q)  # reference predicted
            rouge1f = score['rouge1'].fmeasure
            sim_sum += rouge1f
#         print(q, sim_sum)
        all_q_gens_scores.append(sim_sum)
    # calculate the most similar (representative) question
    max_score = max(all_q_gens_scores)
    q = q_gens[np.argmax(all_q_gens_scores)]
    
    # calculate score of the selected question to the reference question
    score = scorer.score(q_ref, q)  # reference predicted
    scores.append(score['rouge1'].fmeasure)
#     print(q_ref)
    print(q)
#     break

print(scores)
print(np.mean(scores))
#     print('\n')

What are the symptoms of anemia?
What is the cause of anemia?
What is Herbert Spencer known for?
what was the sociology of  Herbert Spencer?
What is ketosis?
What is the ketosis diet?
What are examples of important real-time databases?
What is the database system
What is the ACL injury?
What is the cause of ACL injuries?
Where are turkeys from?
What is the turkeys?
What is the impact of Darwin's theory on modern biology?
What was the theory of evolution?
Tell me about makos sharks.
What is the mako shark
Why are turkeys eaten on Thanksgiving?
What is the turkey?
What is the relationship of Netflix with Blockbuster?
what was the new name of Blockbuster
What are good melatonin sources in food?
What is melatonin?
What are typical pasta dishes?
What is the pasta?
What are the symptoms of lung cancer?
What is the common symptom of lung cancer?
Which Smithsonian museums in Washington D.C. are the most popular?
what is the name of the museum
Tell me about feijoada and its significance.
What i

What is the name of the school?
Tell me about the history of Washington D.C..
What is the Half Smoke?
How is feijoada similar or different from cassoulet?
what is the feijoada?
Is melatonin bad for you?
What is the side effect of melatonin?
What is done with the whey after Parmesan cheese production?
What is the whey
Can lung cancer spread to the throat?
What is the stage of lung cancer?
Why is the National Air and Space Museum important in Washington D.C.?
What is the museum?
Tell me about the author of the Stanford Experiment.
What was the experiment?
What was the first artificial satellite?
What was the first satellite launched?
What are the origins of popular music? 
What is the origin of music?
Why was keto diet original developed?
What is the ketogenic diet?
How is intermittent fasting related to keto?
What else can lectins cause?
How is Herbert Spencer's work related to Comte?
What was the first thing Spencer did?
Can anemia go away?
What is the cause of megaloblastic anemia?
Wh